Simple notebook with data load and few NNs for raw and scattered transformed datas

In [72]:
from google.colab import drive
drive.mount('/content/drive')  

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip -q /content/drive/MyDrive/Gravity_Spy_Glitches_whitened_1.zip
!unzip -q /content/drive/MyDrive/Gravity_Spy_Glitches_whitened_2.zip
!unzip -q /content/drive/MyDrive/Gravity_Spy_Glitches_whitened_3.zip
!unzip -q /content/drive/MyDrive/Gravity_Spy_Glitches_whitened_4.zip
!unzip -q /content/drive/MyDrive/Gravity_Spy_Glitches_whitened_5.zip
!unzip -q /content/drive/MyDrive/Gravity_Spy_Glitches_whitened_6.zip
!unzip -q /content/drive/MyDrive/Gravity_Spy_Glitches_whitened_7.zip 

replace content/sample_data/Gravity_Spy/qiOJvCXifs_whitened.h5? [y]es, [n]o, [A]ll, [N]one, [r]ename: replace content/sample_data/Gravity_Spy_2/uyP3sTlb5y_whitened.h5? [y]es, [n]o, [A]ll, [N]one, [r]ename: replace content/sample_data/Gravity_Spy_3/sQJmXWTpts_whitened.h5? [y]es, [n]o, [A]ll, [N]one, [r]ename: replace content/sample_data/Gravity_Spy_4/vaYysGSAYP_whitened.h5? [y]es, [n]o, [A]ll, [N]one, [r]ename: replace content/sample_data/Gravity_Spy_5/9HA6RTlKNJ_whitened.h5? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [1]:
try:
    from gwpy.timeseries import TimeSeries
except ModuleNotFoundError: 
    !pip install --quiet gwpy
    from gwpy.timeseries import TimeSeries

try:
    from kymatio.torch import Scattering1D
except ModuleNotFoundError: 
    !pip install --quiet kymatio
    from kymatio.torch import Scattering1D

In [2]:
import os
import h5py as h5
import pandas as pd
import matplotlib.pyplot as plt
from os import listdir
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchaudio
import sys
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from torchsummary import summary
import torchvision
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier

In [3]:
from IPython.display import clear_output

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


Dataset Utility

In [5]:
def construct_dataframe(path,t_delta=None):
  name_data=[]
  starting_times_data=[]
  timeseries_data=[]
  type_data=[]
  for file in listdir(path):
    if file != '.ipynb_checkpoints':
        fout=h5.File(path+file,'r') 
    type_of_glitch=list(fout.keys())[0]
    dataset=fout[type_of_glitch]
    t = TimeSeries(dataset[()])
    ln=len(t)/2
    #if no t_delta is provided, the whole the timeseries are not cut 
    if t_delta==None:
      t_delta=ln
    name_data.append(file.split('.')[0].split('_')[0])
    type_data.append(type_of_glitch)
    starting_times_data.append(dataset.attrs['t0'])
    dataset_cut=dataset[()][int(ln-t_delta):int(ln+t_delta)]
    timeseries_data.append(dataset_cut)
    # intialise data of lists.

  data_Virgo = {'ID':name_data,
        'Type_Of_Glitch':type_data,
        'Starting_Times':starting_times_data,
        'TimeSeries':timeseries_data}
    # Create DataFrame
  df= pd.DataFrame(data_Virgo)
  return df

def check_remove_nanDF(dataframe, col='TimeSeries'):
    val=dataframe[col].values
    idx=[]
    for i in range(len(val)):
        if np.isnan(val[i])[0]==True:
            idx+=[i]
    print("removed timeseries:", len(idx))
    print(idx)
    return dataframe.drop(labels=idx,axis=0)


def build_torchDataset(Dataset, norm=True):
    X,y=Dataset.T
    data=[]
    label=[]
    for element in X:
        data+= [element]
    for i in y:
        label+=[i]
    X_t=torch.tensor(np.asarray(data)).unsqueeze(1)
    y_t=torch.tensor(label).long()
    if norm:
        X_t=(X_t-X_t.mean(0))/X_t.std(0)
    return TensorDataset(X_t,y_t)



def build_dataset(path='/content/content/sample_data/', idx=['_2', '_3', '_4', '_5', '_6', '_7']):
    df_LIGO=construct_dataframe(path+'Gravity_Spy/')
    df_LIGO=check_remove_nanDF(df_LIGO)
    for i in idx:
        df_tmp=construct_dataframe(path+'Gravity_Spy'+i+'/')
        df_tmp=check_remove_nanDF(df_tmp)
        df_LIGO=pd.concat([df_LIGO,df_tmp],axis=0)
    return df_LIGO

def pytorch_glitch(df,norm=True):
    df.Type_Of_Glitch = pd.Categorical(df.Type_Of_Glitch)
    df['code']=df.Type_Of_Glitch.cat.codes
    return build_torchDataset(df.drop(columns=['ID', 'Type_Of_Glitch', 'Starting_Times']).values,norm)

def get_labels(df):
    return  sorted(list(set(datapoint[1] for datapoint in df.to_numpy())))

def remove_row_by_freq(df, col='Type_Of_Glitch', freq=50):
    return df[df.groupby(col)[col].transform('count').ge(freq)]


def split_stratified_into_train_val_test(df_input, stratify_colname='Type_Of_Glitch',
                                         frac_train=0.75, frac_val=0.15, frac_test=0.10,
                                         random_state=137):
    '''
    Splits a Pandas dataframe into three subsets (train, val, and test)
    following fractional ratios provided by the user, where each subset is
    stratified by the values in a specific column (that is, each subset has
    the same relative frequency of the values in the column). It performs this
    splitting by running train_test_split() twice.

    Parameters
    ----------
    df_input : Pandas dataframe
        Input dataframe to be split.
    stratify_colname : str
        The name of the column that will be used for stratification. Usually
        this column would be for the label.
    frac_train : float
    frac_val   : float
    frac_test  : float
        The ratios with which the dataframe will be split into train, val, and
        test data. The values should be expressed as float fractions and should
        sum to 1.0.
    random_state : int, None, or RandomStateInstance
        Value to be passed to train_test_split().

    Returns
    -------
    df_train, df_val, df_test :
        Dataframes containing the three splits.
    '''

    if frac_train + frac_val + frac_test != 1.0:
        raise ValueError('fractions %f, %f, %f do not add up to 1.0' % \
                         (frac_train, frac_val, frac_test))

    if stratify_colname not in df_input.columns:
        raise ValueError('%s is not a column in the dataframe' % (stratify_colname))

    X = df_input # Contains all columns.
    y = df_input[[stratify_colname]] # Dataframe of just the column on which to stratify.

    # Split original dataframe into train and temp dataframes.
    df_train, df_temp, y_train, y_temp = train_test_split(X,
                                                          y,
                                                          stratify=y,
                                                          test_size=(1.0 - frac_train),
                                                          random_state=random_state)

    # Split the temp dataframe into val and test dataframes.
    relative_frac_test = frac_test / (frac_val + frac_test)
    df_val, df_test, y_val, y_test = train_test_split(df_temp,
                                                      y_temp,
                                                      stratify=y_temp,
                                                      test_size=relative_frac_test,
                                                      random_state=random_state)

    assert len(df_input) == len(df_train) + len(df_val) + len(df_test)
    return df_train, df_val, df_test

Networks

In [6]:
def evaluate_accuracy(model,transform, dataloader):
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for data in dataloader:
            images, y = data
            if transform:
                images=transform(images.to(device))
            else:
                images=images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)  
            total += y.size(0) 
            correct += (predicted == y.to(device)).sum().item()
    accuracy = 100 * correct / total  
    return accuracy

def accuracy_classes(net,transform,dataloader,classes): 
    correct_pred = {classname: 0 for classname in classes}
    total_pred = {classname: 0 for classname in classes}
    net.eval()
    with torch.no_grad():
        for data in dataloader:
            images, labels = data
            if transform:
                images=transform(images.to(device))
            else:
                images=images.to(device)
            outputs = net(images)
            _, predictions = torch.max(outputs, 1)
            for label, prediction in zip(labels.to(device), predictions):
                if label == prediction:
                    correct_pred[classes[label]] += 1
                total_pred[classes[label]] += 1

    for classname, correct_count in correct_pred.items():
        accuracy = 100 * float(correct_count) / total_pred[classname]
        print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')


def training(net,transform,optimizer,criterion,train_loader,test_loader,num_epochs):
  history={'loss':[],'train_accuracy':[],'val_accuracy':[]}
  for epoch in tqdm(range(num_epochs)):
      running_loss = 0.0
      running_corrects = 0.0
      net.train()
      for i, data in enumerate(train_loader, 0):
          inputs, y = data
          optimizer.zero_grad()
          if transform:
            inputs=transform(inputs.to(device))
          else:
            inputs=inputs.to(device)
          outputs = net(inputs)

          loss = criterion(outputs, y.to(device))
          loss.backward()
          optimizer.step()
 
          _, predicted = torch.max(outputs.data.cpu(), 1)
          running_corrects += torch.sum(predicted == y.data).item()
        
          running_loss += loss.item() * inputs.size(0)
        

      epoch_loss = running_loss / len(train_loader.dataset)
      epoch_acc = 100.0*running_corrects / len(train_loader.dataset)
      val_acc=evaluate_accuracy(net,transform,test_loader)
      history['loss'].append(epoch_loss)
      history['train_accuracy'].append(epoch_acc)
      history['val_accuracy'].append(val_acc)
      print('Epoch {} Loss: {:.4f} Acc: {:.4f}% Val Acc: {:.4f}%'.format(epoch+1, epoch_loss, epoch_acc, val_acc))  
  return history


class LogisticRegression(nn.Module):
    def __init__(self, n_input, n_output=18):
        super().__init__()
        self.fc1 = nn.Linear(n_input, n_output)
    def forward(self, x):
        x=x.view(x.shape[0],-1)
        x = self.fc1(x)
        return F.log_softmax(x, dim=1).squeeze()

class M5(nn.Module):
    def __init__(self, n_input=1, n_output=35, stride=16, n_channel=32):
        super().__init__()
        self.conv1 = nn.Conv1d(n_input, n_channel, kernel_size=80, stride=stride)
        self.bn1 = nn.BatchNorm1d(n_channel)
        self.pool1 = nn.MaxPool1d(4)
        self.conv2 = nn.Conv1d(n_channel, n_channel, kernel_size=3)
        self.bn2 = nn.BatchNorm1d(n_channel)
        self.pool2 = nn.MaxPool1d(4)
        self.conv3 = nn.Conv1d(n_channel, 2 * n_channel, kernel_size=3)
        self.bn3 = nn.BatchNorm1d(2 * n_channel)
        self.pool3 = nn.MaxPool1d(4)
        self.conv4 = nn.Conv1d(2 * n_channel, 2 * n_channel, kernel_size=3)
        self.bn4 = nn.BatchNorm1d(2 * n_channel)
        self.pool4 = nn.MaxPool1d(4)
        self.fc1 = nn.Linear(2 * n_channel, n_output)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(self.bn1(x))
        x = self.pool1(x)
        x = self.conv2(x)
        x = F.relu(self.bn2(x))
        x = self.pool2(x)
        x = self.conv3(x)
        x = F.relu(self.bn3(x))
        x = self.pool3(x)
        x = self.conv4(x)
        x = F.relu(self.bn4(x))
        x = self.pool4(x)
        x = F.avg_pool1d(x, x.shape[-1])
        x = x.permute(0, 2, 1)
        x = self.fc1(x)
        return F.log_softmax(x, dim=2).squeeze()


class LogScattering:
    def __init__(self,J,T,Q, sample_rate=16000,device='cuda'):
        self.transform= torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=T).to(device)
        self.scattering = Scattering1D(J, T, Q).to(device)
        self.eps=1e-6
    def __call__(self,x):
        x /= torch.max(torch.abs(x))
        x=self.transform(x.squeeze())
        x=self.scattering(x.contiguous())
        return torch.log(torch.abs(x[:,1:,:])+self.eps).mean(-1)

        
class Wavelet:
    def __init__(self, J, T, Q,image_size = 64,sample_rate=16000,device='cuda'):
        '''
        This class does a sequence of transformations.
        1- waveform are reshaped from smaple_rate to T
        2-wavelet scattering transform is then applied to the reshaped signals
        3-the transformed samples are reshaped to a (image_size,image_size)
        4-Finally, batch normalization is applied to the reshaped images
        '''
        self.scattering = Scattering1D(J, T, Q).to(device)
        self.resize = torchvision.transforms.Resize((image_size,image_size)).to(device)
        self.transform= torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=T).to(device)
        self.bn = nn.BatchNorm2d(1).to(device)

    def __call__(self, x):
        x=self.transform(x) #step 1
        #x=x/ torch.max(torch.abs(x))
        x=self.scattering(x.contiguous()) #step 2
        x=self.resize(x) #step 3
        x=self.bn(x) #step 4
        return x
     

class Flatten(nn.Module):
    def forward(self, input):
        batch_size = input.size(0)
        out = input.view(batch_size,-1)
        return out # (batch_size, *size)

#General convolutional neural network for squared images with size=(img_shape,img_shape)
#The list hidden_sizes fix the numbers of convolutional layers (len(hidden_sizes))
#And the kernel for the ith layer (hidden_sizes[i])
#The activation functions can be choosen between relu, elu and tanh
#Dropout=True ad a dropout to each convolutional layers with porbability p
class WaveletCNN(nn.Module):
    def __init__(self, img_shape, hidden_sizes=[16],dropout=True,p=0.05,act='relu',input_channel=1,n_classes=35):
        super(WaveletCNN, self).__init__()

        net=[]
        hidden_sizes=[input_channel]+hidden_sizes
        for i in range(len(hidden_sizes)-1):
          net.append(nn.Conv2d(hidden_sizes[i], hidden_sizes[i+1], kernel_size=3, padding=1))
          net.append(nn.BatchNorm2d(hidden_sizes[i+1]))
          if act=='relu':
            net.append(nn.ReLU())
          elif act=='elu':
            net.append(nn.SiLU())
          else:
            net.append(nn.Tanh())
          if dropout:
            net.append(nn.Dropout(p=p))
          net.append(nn.MaxPool2d(kernel_size=2, stride=2))

        net.append(Flatten())
        net.append(nn.Linear(int(hidden_sizes[-1] * (img_shape/(2**(len(hidden_sizes)-1)))**2), n_classes))
        self.net=nn.ModuleList(net)

        
    def forward(self, x):
        for l in self.net:
          x=l(x)
        return x
     


In [26]:
df_LIGO=build_dataset()

removed timeseries: 4
[317, 326, 447, 965]
removed timeseries: 31
[2, 20, 35, 46, 92, 148, 154, 170, 212, 214, 216, 271, 274, 329, 364, 367, 383, 437, 442, 467, 473, 552, 572, 578, 583, 634, 644, 662, 775, 797, 837]
removed timeseries: 24
[17, 23, 76, 117, 147, 154, 165, 285, 286, 344, 430, 453, 454, 511, 550, 555, 567, 582, 616, 647, 696, 702, 728, 748]
removed timeseries: 12
[29, 72, 73, 96, 99, 105, 116, 163, 372, 580, 667, 764]
removed timeseries: 21
[79, 115, 151, 162, 183, 204, 337, 339, 368, 371, 375, 383, 434, 454, 482, 541, 542, 560, 644, 684, 686]
removed timeseries: 17
[60, 210, 217, 227, 252, 342, 347, 382, 405, 445, 454, 459, 495, 552, 575, 636, 822]
removed timeseries: 27
[21, 50, 58, 76, 78, 94, 110, 112, 122, 127, 174, 260, 345, 418, 429, 484, 512, 689, 696, 734, 771, 774, 778, 781, 817, 829, 832]


In [11]:
df_LIGO['Type_Of_Glitch'].value_counts()

Blip                   1707
Koi_Fish                598
Low_Frequency_Burst     557
Power_Line              432
Low_Frequency_Lines     416
Scattered_Light         415
Extremely_Loud          287
Scratchy                247
Light_Modulation        221
Whistle                 144
1080Lines               140
No_Glitch               137
Tomte                   100
Repeating_Blips          79
1400Ripples              75
Chirp                    53
Air_Compressor           51
None_of_the_Above        40
Paired_Doves             15
Wandering_Line            6
Violin_Mode               2
Name: Type_Of_Glitch, dtype: int64

In [ ]:
rem=True
if rem:
  df_LIGO=remove_row_by_freq(df_LIGO,freq=400)

In [ ]:
df_LIGO['Type_Of_Glitch'].value_counts()

Blip                   1707
Koi_Fish                598
Low_Frequency_Burst     557
Power_Line              432
Low_Frequency_Lines     416
Scattered_Light         415
Name: Type_Of_Glitch, dtype: int64

In [ ]:
labels=get_labels(df_LIGO)
labels

['Blip',
 'Koi_Fish',
 'Low_Frequency_Burst',
 'Low_Frequency_Lines',
 'Power_Line',
 'Scattered_Light']

In [ ]:
train_set, val_set, test_set = split_stratified_into_train_val_test(df_LIGO)

In [ ]:
y_train=list(train_set['Type_Of_Glitch'])
#X_train=np.array(train_set['TimeSeries'])
X_train=list(train_set['TimeSeries'])

In [ ]:
X_train

[array([ 61.98176 , -44.731094,  53.225513, ..., -21.75924 ,
         32.1147  , -14.386234], dtype=float32),
 array([  8.856661, -31.338194,  21.287683, ..., -51.251   ,
         15.913797, -35.896572], dtype=float32),
 array([-104.736664,   67.95733 ,  -94.91456 , ...,   14.71714 ,
         -19.659868,   13.33982 ], dtype=float32),
 array([ 18.633017  , -32.828953  ,  18.63423   , ...,
          9.599443  ,   0.69563925,   6.4740677 ], dtype=float32),
 array([ -6.393942 ,  13.459466 ,  -3.4923527, ...,  22.15883  ,
        -11.374364 ,  21.953485 ], dtype=float32),
 array([ 33.605152 , -22.073002 ,  27.484756 , ...,  -8.680712 ,
         31.363413 ,   3.1530616], dtype=float32),
 array([-36.181843, -22.857513, -27.48172 , ..., -36.695473,
          9.354273, -44.366848], dtype=float32),
 array([  3.4932518, -11.713887 ,  -1.2306863, ...,  -6.0563374,
         -2.049277 ,  -3.0569465], dtype=float32),
 array([-14.243475,  11.928955, -11.245685, ...,  11.407271,
        -16.439955,   8

In [ ]:
clf = DecisionTreeClassifier(min_samples_split=15,min_samples_leaf=8)
clf.fit(train_set)

TypeError: ignored

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_set['Type_Of_Glitch']=le.fit_transform(train_set['Type_Of_Glitch'])

In [ ]:
y_train=train_set['Type_Of_Glitch']
X_train=train_set['TimeSeries']

y_train=np.array(y_train)
#X_train=np.array(X_train)

In [ ]:
list(X_train)

[array([  3.6632323, -14.314723 ,   1.458568 , ...,  35.1388   ,
        -23.360325 ,  33.19297  ], dtype=float32),
 array([ 1.4285746 ,  0.7284764 ,  3.8203614 , ..., -1.0152506 ,
         0.61210847,  0.84220004], dtype=float32),
 array([-55.240734,  47.049988, -36.359932, ..., -14.404697,
         -8.706397, -10.918415], dtype=float32),
 array([  1.1522088,  -1.9033073,  -4.737227 , ..., -19.174288 ,
         22.397442 , -18.407171 ], dtype=float32),
 array([  24.833828,  193.66808 ,   35.820103, ..., -315.94745 ,
         309.8678  , -336.4164  ], dtype=float32),
 array([-7.5042295,  3.3780255, -1.9080262, ..., -6.4139795,
         5.5610733, -2.8226871], dtype=float32),
 array([  8.548874 , -92.695465 ,  49.136013 , ..., -11.401145 ,
          1.3648477,  -4.05884  ], dtype=float32),
 array([-27.6199  ,  18.973608, -17.427742, ..., -28.685934,
         20.376457, -23.677866], dtype=float32),
 array([-14.411546  ,   0.92108035, -12.075314  , ...,
         12.349858  ,  -4.8455634 ,

In [ ]:
clf = DecisionTreeClassifier(min_samples_split=15,min_samples_leaf=8)
clf.fit(X_train,y_train)

ValueError: ignored

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

In [ ]:
iris = load_iris()
print(len(iris.target))
len(iris.data)

150


150

In [ ]:
train_set=pytorch_glitch(train_set)
val_set=pytorch_glitch(val_set)
#test_set=pytorch_glitch(test_set)

In [ ]:
len(train_set)

3093

In [ ]:
len(val_set)

619

In [ ]:
len(test_set)

413

In [ ]:
sample_rate=34864

In [ ]:
batch_size = 32

if device == "cuda":
    num_workers = 1
    pin_memory = True
else:
    num_workers = 0
    pin_memory = False

train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    #collate_fn=collate_fn,
    pin_memory=pin_memory,
)
test_loader = torch.utils.data.DataLoader(
    val_set,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    num_workers=num_workers,
    #collate_fn=collate_fn,
    pin_memory=pin_memory,
)

#Raw Data

In [ ]:
T=sample_rate

In [ ]:
transform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=T).to(device)

In [ ]:
len(labels)

21

In [ ]:
model = LogisticRegression(n_input=T, n_output=len(labels)).to(device)    

In [ ]:
model = M5( n_output=len(labels)).to(device)

In [ ]:
criterion = nn.NLLLoss()

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=0.01)

In [ ]:
summary(model,(1,T))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 21]         732,165
Total params: 732,165
Trainable params: 732,165
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.13
Forward/backward pass size (MB): 0.00
Params size (MB): 2.79
Estimated Total Size (MB): 2.93
----------------------------------------------------------------


In [ ]:
num_epochs=500

In [ ]:
history=training(model,transform,optimizer,criterion,train_loader,test_loader,num_epochs)

In [ ]:
accuracy_classes(model,transform,train_loader,labels)

In [ ]:
accuracy_classes(model,transform,test_loader,labels)

# LOGSCATTERING

In [ ]:
T = sample_rate
J = 8
Q = 1

In [ ]:
transform=LogScattering(J,T,Q,sample_rate=sample_rate) #TO do: preprocess the dataset and not the minibatch
test=transform(torch.randn(256,1,sample_rate).to(device))
test.shape

torch.Size([256, 42])

In [ ]:
net= LogisticRegression(n_input=test.shape[-1], n_output=len(labels)).to(device) 

In [ ]:
num_epochs = 100

In [ ]:
summary(net,(1,test.shape[-1]))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                [-1, 1, 18]             774
Total params: 774
Trainable params: 774
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [ ]:
net(test).shape

torch.Size([256, 18])

In [ ]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(net.parameters(), lr=0.005, weight_decay=0.0001)

In [ ]:
history=training(net,transform,optimizer,criterion,train_loader,test_loader,num_epochs)

# Wavenet

In [ ]:
net=WaveletCNN(64,hidden_sizes=[16],n_classes=len(labels)).to(device)
     
T = 8000
J = 4
Q = 4

wvlt=Wavelet(J,T,Q,64,sample_rate=sample_rate,device=device)
     
summary(net,(1,64,64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 64, 64]             160
       BatchNorm2d-2           [-1, 16, 64, 64]              32
              ReLU-3           [-1, 16, 64, 64]               0
           Dropout-4           [-1, 16, 64, 64]               0
         MaxPool2d-5           [-1, 16, 32, 32]               0
           Flatten-6                [-1, 16384]               0
            Linear-7                    [-1, 6]          98,310
Total params: 98,502
Trainable params: 98,502
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.02
Forward/backward pass size (MB): 2.25
Params size (MB): 0.38
Estimated Total Size (MB): 2.64
----------------------------------------------------------------


In [ ]:
num_epochs = 100
learning_rate = 0.01
     

criterion = nn.CrossEntropyLoss() #Another option is to use NegativeLog likelihood in combination with log-sofmatx applied to the NNs output
optimizer = optim.Adam(net.parameters(), lr=learning_rate,weight_decay=1e-4)

In [ ]:
history=training(net,wvlt,optimizer,criterion,train_loader,test_loader,num_epochs)

  0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch 1 Loss: 10.5693 Acc: 33.3980% Val Acc: 39.5800%
Epoch 2 Loss: 2.3134 Acc: 44.0026% Val Acc: 44.2649%
Epoch 3 Loss: 1.7231 Acc: 48.5289% Val Acc: 46.2036%
Epoch 4 Loss: 1.6107 Acc: 49.6929% Val Acc: 48.3037%
Epoch 5 Loss: 1.4930 Acc: 51.1154% Val Acc: 48.7884%
Epoch 6 Loss: 1.3366 Acc: 53.5403% Val Acc: 47.3344%
Epoch 7 Loss: 1.3408 Acc: 55.0275% Val Acc: 50.5654%
Epoch 8 Loss: 1.2988 Acc: 54.0252% Val Acc: 53.7964%
Epoch 9 Loss: 1.2374 Acc: 57.6463% Val Acc: 53.4733%
Epoch 10 Loss: 1.1978 Acc: 58.1636% Val Acc: 49.7577%
Epoch 11 Loss: 1.1894 Acc: 57.4846% Val Acc: 54.4426%
Epoch 12 Loss: 1.2043 Acc: 58.5839% Val Acc: 52.8271%
Epoch 13 Loss: 1.1477 Acc: 60.6531% Val Acc: 50.5654%
Epoch 14 Loss: 1.1341 Acc: 61.2350% Val Acc: 49.9192%
Epoch 15 Loss: 1.1063 Acc: 60.7501% Val Acc: 55.4120%
Epoch 16 Loss: 1.0477 Acc: 63.1102% Val Acc: 54.4426%
Epoch 17 Loss: 1.0055 Acc: 65.1471% Val Acc: 55.5735%
Epoch 18 Loss: 1.0396 Acc: 64.3388% Val Acc: 56.7044%
Epoch 19 Loss: 1.0152 Acc: 63.5952% 

In [ ]:
accuracy_classes(net,wvlt,train_loader,labels)

Accuracy for class: Blip  is 90.2 %
Accuracy for class: Koi_Fish is 81.0 %
Accuracy for class: Low_Frequency_Burst is 83.7 %
Accuracy for class: Low_Frequency_Lines is 92.3 %
Accuracy for class: Power_Line is 83.0 %
Accuracy for class: Scattered_Light is 92.6 %


In [ ]:
accuracy_classes(net,wvlt,test_loader,labels)

Accuracy for class: Blip  is 66.0 %
Accuracy for class: Koi_Fish is 54.4 %
Accuracy for class: Low_Frequency_Burst is 39.8 %
Accuracy for class: Low_Frequency_Lines is 35.5 %
Accuracy for class: Power_Line is 30.8 %
Accuracy for class: Scattered_Light is 63.5 %


# Implement a Decision Tree

In [8]:
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

Choose to cleassify only three signals

In [27]:
rem=True
if rem:
  df_LIGO=remove_row_by_freq(df_LIGO,freq=500)

In [28]:
df_LIGO['Type_Of_Glitch'].value_counts()

Blip                   1707
Koi_Fish                598
Low_Frequency_Burst     557
Name: Type_Of_Glitch, dtype: int64

In [29]:
labels=get_labels(df_LIGO)
labels

['Blip', 'Koi_Fish', 'Low_Frequency_Burst']

In [30]:
train_set, val_set, test_set = split_stratified_into_train_val_test(df_LIGO)

In [31]:
train_set

,ID,Type_Of_Glitch,Starting_Times,TimeSeries
134,mCfkxA2uG9,Koi_Fish,1.135818e+09,"[3.1578147, 6.000953, 2.1760805, 6.1133423, 2...."
13,dx7WHDdb1h,Blip,1.128896e+09,"[17.248617, -15.825822, 17.081852, -16.945065,..."
14,zQFycHixNc,Blip,1.126845e+09,"[-9.454135, 3.2714403, -1.7350987, 5.5284967, ..."
683,B0hr95jPJX,Low_Frequency_Burst,1.134473e+09,"[20.339596, -38.858837, 15.804584, -35.188015,..."
334,vbKJPF1cAX,Blip,1.129089e+09,"[-3.6846058, 1.1422064, -4.7741756, 4.628877, ..."
...,...,...,...,...
449,FQRFlEsQw2,Koi_Fish,1.127133e+09,"[-26.002415, 40.329105, -22.20381, 42.614117, ..."
354,fOJ0hbuURb,Blip,1.132608e+09,"[11.165221, -14.541635, 15.232593, -23.608643,..."
103,YlFmohXD2c,Blip,1.130526e+09,"[-4.618989, -2.7429163, -1.5238941, 2.1182494,..."
619,d53Z7rQZO7,Blip,1.132885e+09,"[-36.713657, -7.403705, -38.781494, -0.2375935..."


In [32]:
val_set

,ID,Type_Of_Glitch,Starting_Times,TimeSeries
861,s53vlyTTmN,Blip,1.133272e+09,"[46.84929, -96.00296, 44.107574, -84.269165, 4..."
599,TLRVooQ9ER,Blip,1.126547e+09,"[9.876681, 3.742105, 11.233343, -15.196399, 4...."
349,yXpeaLquRv,Low_Frequency_Burst,1.135054e+09,"[4.3103976, -2.8278012, 0.43318087, -2.5168357..."
78,YVG2onUXP8,Blip,1.127479e+09,"[5.2311277, -5.534153, 9.809488, -11.54866, 7...."
132,HQ2eq6sSRC,Koi_Fish,1.129161e+09,"[-5.947146, 5.626959, -6.963896, 3.7957323, -0..."
...,...,...,...,...
474,6TCrjF1XTB,Blip,1.130629e+09,"[-4.790031, 1.406966, -6.6568685, 5.0377693, -..."
302,2t3CDwuqop,Blip,1.132680e+09,"[-1.2673815, 7.041681, -5.559562, 8.506424, -3..."
575,JBgxaDlTUR,Koi_Fish,1.132674e+09,"[38.3718, -38.77911, 40.980637, -44.452827, 40..."
262,2L70tspbqY,Koi_Fish,1.133794e+09,"[11.323378, -10.676059, 13.215044, -8.266748, ..."


In [33]:
test_set

,ID,Type_Of_Glitch,Starting_Times,TimeSeries
334,s8udTaWqxU,Blip,1.128003e+09,"[3.372138, 7.1140313, -0.8420415, -1.034482, -..."
685,Mwnk1Y46iQ,Low_Frequency_Burst,1.134917e+09,"[-50.385975, 18.321331, -40.514786, 29.911003,..."
338,JcUnDiI6Xf,Blip,1.134148e+09,"[14.492058, -9.80689, 7.4807644, -8.106958, 12..."
243,bHn40MaTxZ,Blip,1.137010e+09,"[-5.3881636, 2.060703, -5.906797, 0.200521, -3..."
682,wpqkFZhk1l,Blip,1.126434e+09,"[-7.176301, 10.854085, -8.453667, 16.832098, -..."
...,...,...,...,...
609,0ljBnkXHQy,Blip,1.132759e+09,"[2.8560731, -1.0014334, 0.87287503, 1.1784528,..."
836,TmdnzV5jWL,Koi_Fish,1.130976e+09,"[3.5065563, -0.69406515, 4.589477, -3.8399892,..."
250,R3mgmeGYSP,Low_Frequency_Burst,1.135596e+09,"[18.024744, -16.955997, 10.607194, -23.763557,..."
311,Snffa3eyBl,Koi_Fish,1.129737e+09,"[33.786015, -88.07689, 73.02463, 22.677265, -2..."


In [34]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_set['Type_Of_Glitch']=le.fit_transform(train_set['Type_Of_Glitch'])
test_set['Type_Of_Glitch']=le.fit_transform(test_set['Type_Of_Glitch'])

In [35]:
train_set

,ID,Type_Of_Glitch,Starting_Times,TimeSeries
134,mCfkxA2uG9,1,1.135818e+09,"[3.1578147, 6.000953, 2.1760805, 6.1133423, 2...."
13,dx7WHDdb1h,0,1.128896e+09,"[17.248617, -15.825822, 17.081852, -16.945065,..."
14,zQFycHixNc,0,1.126845e+09,"[-9.454135, 3.2714403, -1.7350987, 5.5284967, ..."
683,B0hr95jPJX,2,1.134473e+09,"[20.339596, -38.858837, 15.804584, -35.188015,..."
334,vbKJPF1cAX,0,1.129089e+09,"[-3.6846058, 1.1422064, -4.7741756, 4.628877, ..."
...,...,...,...,...
449,FQRFlEsQw2,1,1.127133e+09,"[-26.002415, 40.329105, -22.20381, 42.614117, ..."
354,fOJ0hbuURb,0,1.132608e+09,"[11.165221, -14.541635, 15.232593, -23.608643,..."
103,YlFmohXD2c,0,1.130526e+09,"[-4.618989, -2.7429163, -1.5238941, 2.1182494,..."
619,d53Z7rQZO7,0,1.132885e+09,"[-36.713657, -7.403705, -38.781494, -0.2375935..."


Define the training and target variables for the Tree Classifier

In [36]:
y_train=train_set['Type_Of_Glitch']
X_train=train_set['TimeSeries']

X_test=test_set['TimeSeries']
y_test=test_set['Type_Of_Glitch']

#y_train=np.array(y_train)
#X_train=np.array(X_train)

Turn tthe X_train and y_train into arrays (a bit of a hack, probably there are better ways to do this)

In [37]:
X_train_array=[]
i=0
for i in range(len(X_train)):
  a=X_train.iloc[i].tolist()
  a=np.array(a)
  X_train_array.append(a)
y_train=np.array(y_train)

Set up the Tree Classifier and evaluate its performance 

In [38]:
clf = DecisionTreeClassifier(random_state=0)
#cross_val_score(clf, X_train_array, y_train, cv=10)

In [39]:
clf.fit(X_train_array, y_train)

DecisionTreeClassifier(random_state=0)

In [40]:
print(clf.get_depth())
clf.get_n_leaves()

21


192

In [41]:
X_test_array=[]
i=0
for i in range(len(X_test)):
  a=X_test.iloc[i].tolist()
  a=np.array(a)
  X_test_array.append(a)
y_test=np.array(y_test)

In [42]:
print(y_test)
predicted_y=clf.predict(X_test_array)

[0 2 0 ... 2 1 0]


In [43]:
diff_array=y_test-predicted_y
errs_array=[]
for i in diff_array:
  if i!=0:
    errs_array.append(i)

The performance is quite poor.

In [44]:
len(errs_array)/len(diff_array)

0.36585365853658536

# Gradient Boosted Decision Tree

In [45]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

How do we set the number of estimators? We kept the same ratio as in [this example](https://scikit-learn.org/stable/auto_examples/ensemble/plot_adaboost_multiclass.html#sphx-glr-auto-examples-ensemble-plot-adaboost-multiclass-py) .

In [46]:
300/13000*len(X_test_array)

6.6230769230769235

In [30]:
#clf_boosted=AdaBoostClassifier(
#    clf, n_estimators=7, learning_rate=1,
#    algorithm="SAMME",
#)

#clf_boosted.fit(X_train_array, y_train)
#predicted_y=clf_boosted.predict(X_test_array)

#diff_array=y_test-predicted_y
#errs_array=[]
#for i in diff_array:
#  if i!=0:
#    errs_array.append(i)

In [34]:
len(errs_array)/len(diff_array)

0.36585365853658536

In [47]:
clf = GradientBoostingClassifier(n_estimators=50, learning_rate=1.0, max_depth=1, random_state=0).fit(X_train_array, y_train)
clf.fit(X_train_array, y_train)
predicted_y=clf.predict(X_test_array)
diff_array=y_test-predicted_y

errs_array=[]
for i in diff_array:
  if i!=0:
    errs_array.append(i)

In [48]:
len(errs_array)/len(diff_array)

0.30662020905923343